# **Initial Setup of Portfolio**

In [86]:
from google.colab import drive

drive.mount('/content/gdrive')

save_path = "/content/gdrive/MyDrive/XML Qns/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [87]:
!pip install selenium
from selenium import webdriver
!apt-get install chromium-driver
!pip install pyodbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [88]:
import pandas as pd
import time
import re
import os
import pyodbc
import pytz
import numpy as np
import statistics
from math import *
from google.colab import files
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [89]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument("--no-sandbox")
  options.add_argument("--headless")
  options.add_argument("--disable-gpu")
  options.add_argument("--window-soze = 1920, 1200")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options = options)
  return driver

driver = web_driver()

def N(x):
  #'Cumulative distribution function for the standard normal distribution'
  return (1.0 + erf(x / sqrt(2.0))) / 2.0

def deltac(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  return np.exp(-y * τ) * N(d1)

def c(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  d2 = d1 - σ * np.sqrt(τ)
  return S * np.exp(-y * (τ)) * N(d1) - K * np.exp(-r * (τ)) * N(d2)

In [90]:
#stock to search
option = 'NVDA250117C00140000'
ticker = 'NVDA'
r = 0.0034 #risk free sg rate as of oct 2024

In [91]:
url = "https://sg.finance.yahoo.com/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.1)

# get search button element
search = driver.find_element(By.XPATH, '//*[@id="header-quote-lookup"]/div/form/input')
# create action chain object
action = ActionChains(driver)
# click the item
action.click(on_element = search)
# perform the operation
action.perform()


#search ticker
search.send_keys(ticker)
search.send_keys(Keys.ENTER)
time.sleep(0.7)

S = float(driver.find_elements(By.XPATH, '/html/body/div[2]/main/section/section/section/article/section[1]/div[2]/div[1]/section/div/section[1]/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

y = float(re.findall(r'\(.*?\)', driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()[27])[0][1:-2]) / 100

print("Stock Price:", S)
print("Dividend Yield:", y)

Stock Price: 141.54
Dividend Yield: 0.0003


Calculate 20 day annualized volatility

In [92]:
def vol():
  # get historical data for volatility
  historical = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/aside/section/nav/ul/li[5]/a')
  # create action chain object
  action = ActionChains(driver)
  # click the item
  action.click(on_element = historical)
  # perform the operation
  action.perform()

  time.sleep(0.7)

  #20 day average volatility
  days = 20

  historical_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[1]/div[3]/table')[0].text.splitlines()

  vol_row_log = []
  Y = []

  for i in range(days):
    adj_close = float((historical_table[i + 3].split())[-2])
    vol_row_log.append(log(adj_close))

  for i in range(days - 1):
    Y.append(vol_row_log[i] - vol_row_log[i+1])

  Y_avg = statistics.fmean(Y)

  #get annual volatility
  vol = sqrt(sum((np.array(Y) - Y_avg)**2) / (days - 1)) * sqrt(252)

  return vol

vol = vol()

print("20 day volatility", vol)

20 day volatility 0.3754370789925649


In [93]:
#go to options page
url = "https://sg.finance.yahoo.com/quote/"+ option + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.3)

option_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()

K = float(option_table[9])
maturity = datetime.combine(datetime.strptime(option_table[11], '%Y-%m-%d').date(), datetime.strptime("11:59:00", '%H:%M:%S').time()) #add 11:59:00 to maturity date
print(option_table)

call_price = float(driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/section[1]/div[2]/div[1]/section/div/section/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

print("Strike_Price:", K)
print("Call_Price:", call_price)
print("Maturity:", maturity)

['Previous close', '15.15', 'Open', '15.60', 'Bid', '15.65', 'Ask', '15.80', 'Strike', '140.00', 'Expiry date', '2025-01-17', "Day's range", '15.35 - 17.28', 'Contract range', '--', 'Volume', '2,728', 'Open interest', '69.02k']
Strike_Price: 140.0
Call_Price: 15.75
Maturity: 2025-01-17 11:59:00


In [94]:
# get_url = driver.current_url
# print("The current url is:"+str(get_url))

The current url is:https://sg.finance.yahoo.com/quote/NVDA250117C00140000/


In [95]:
#current time
date = datetime.now(tz=pytz.timezone('America/New_York')).replace(microsecond = 0).replace(tzinfo=None)
print("NYC current date and time:", date)

time_to_maturity = maturity - date
print("Time to maturity:", time_to_maturity)
print(time_to_maturity.days, "days", time_to_maturity.seconds, "seconds")

#time to maturity (seconds)
tau_seconds = float(time_to_maturity.days) * 24 * 3600 + float(time_to_maturity.seconds)
print(tau_seconds, "seconds")

#time to maturity (years)
tau_years = tau_seconds / (365 * 24 * 3600)
print(tau_years, "years")

NYC current date and time: 2024-10-26 01:42:24
Time to maturity: 83 days, 10:16:36
83 days 36996 seconds
7208196.0 seconds
0.22857039573820395 years


In [96]:
bsm_price = c(S, K, r, y, tau_years, vol)
print("Call_Price:", call_price)
print("BSM_Price:", bsm_price)

Call_Price: 15.75
BSM_Price: 10.903056862065256


In [97]:
#number of shares to hold
delta = deltac(S, K, r, y, tau_years, vol)
print("Delta:", delta)

#value of shares
value_of_shares = delta * S
print("Value of shares:", value_of_shares)

#number of shares to buy
print("Buy", delta, "shares of stock")

#amount to borrow from bank
difference = delta * S - call_price
print("Long difference of", delta, "*", S, "-", call_price, "from bank =", -difference)

Delta: 0.5612596098725205
Value of shares: 79.44068518135656
Buy 0.5612596098725205 shares of stock
Long difference of 0.5612596098725205 * 141.54 - 15.75 from bank = -63.69068518135656


In [103]:
#wealth
portfolio = delta * S - difference
print("Value of Replicating Portfolio:", portfolio)

#profit
profit = float(format(0, '.10f'))
print("Profit:", profit)

Value of Replicating Portfolio: 15.75
Profit: 0.0


In [104]:
#create dataframe
df = pd.DataFrame()

#create new portfolio entry
row = [date, maturity, S, K, difference, r, y, vol, bsm_price, call_price, delta, portfolio, profit, profit]

df = pd.concat([df, pd.DataFrame([row])],axis=0, ignore_index=True)

df.columns = ['Date_Time', 'Maturity', 'Stock_Price', 'Strike_Price', 'Amount_Borrowed_from_Bank', 'Risk_free_rate', 'Dividend_Yield', 'Vol', 'BSM_price', 'Call_Price', 'Delta', 'Wealth', 'Profit_without_Hedging', 'Profit_with_Hedging']

df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,Risk_free_rate,Dividend_Yield,Vol,BSM_price,Call_Price,Delta,Wealth,Profit_without_Hedging,Profit_with_Hedging
0,2024-10-26 01:42:24,2025-01-17 11:59:00,141.54,140.0,63.690685,0.0034,0.0003,0.375437,10.903057,15.75,0.56126,15.75,0.0,0.0


In [101]:
#export csv file
df.to_csv('NVDA.csv', date_format='%Y-%m-%d %H:%M:%S', index = False)
files.download("NVDA.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **To be ran every 5 minutes:**

In [ ]:
#stock to search
option = 'NVDA250117C00140000'
ticker = 'NVDA'
r = 0.0034 #risk free sg rate as of oct 2024

In [105]:
import pandas as pd
import time
import re
import os
import pyodbc
import pytz
import numpy as np
import statistics
from math import *
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument("--no-sandbox")
  options.add_argument("--headless")
  options.add_argument("--disable-gpu")
  options.add_argument("--window-soze = 1920, 1200")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options = options)
  return driver

driver = web_driver()

def N(x):
  return (1.0 + erf(x / sqrt(2.0))) / 2.0

def deltac(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  return np.exp(-y * τ) * N(d1)

def c(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  d2 = d1 - σ * np.sqrt(τ)
  return S * np.exp(-y * (τ)) * N(d1) - K * np.exp(-r * (τ)) * N(d2)

In [121]:
df = pd.read_csv(save_path + "NVDA.csv")
df.head()

#df = pd.read_csv('/home/briansim74/Desktop/NVDA.csv')

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,Risk_free_rate,Dividend_Yield,Vol,BSM_price,Call_Price,Delta,Wealth,Profit_without_Hedging,Profit_with_Hedging
0,2024-10-26 00:31:21,2025-01-17 11:59:00,141.54,140.0,63.691216,0.0034,0.0003,0.375437,10.906046,15.75,0.561263,15.75,0.0,0.0


In [107]:
#go to NVDA stock page
url = "https://sg.finance.yahoo.com/quote/"+ ticker + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.7)

S = float(driver.find_elements(By.XPATH, '/html/body/div[2]/main/section/section/section/article/section[1]/div[2]/div[1]/section/div/section[1]/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

y = float(re.findall(r'\(.*?\)', driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()[27])[0][1:-2]) / 100

print("Stock Price:", S)
print("Dividend Yield:", y)

Stock Price: 141.54
Dividend Yield: 0.0003


In [108]:
def vol():
  # get historical data for volatility
  historical = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/aside/section/nav/ul/li[5]/a')
  # create action chain object
  action = ActionChains(driver)
  # click the item
  action.click(on_element = historical)
  # perform the operation
  action.perform()

  time.sleep(0.7)

  #20 day average volatility
  days = 20

  historical_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[1]/div[3]/table')[0].text.splitlines()

  vol_row_log = []
  Y = []

  for i in range(days):
    adj_close = float((historical_table[i + 3].split())[-2])
    vol_row_log.append(log(adj_close))

  for i in range(days - 1):
    Y.append(vol_row_log[i] - vol_row_log[i+1])

  Y_avg = statistics.fmean(Y)

  #get annual volatility
  vol = sqrt(sum((np.array(Y) - Y_avg)**2) / (days - 1)) * sqrt(252)

  return vol

In [109]:
date = datetime.now(tz=pytz.timezone('America/New_York')).replace(microsecond = 0).replace(tzinfo=None).date()

previous_date = datetime.strptime(df['Date_Time'].iloc[-1], '%Y-%m-%d %H:%M:%S').date()

if(date == previous_date):
  vol = df['Vol'].iloc[-1]
else:
  vol = vol()

print("20 day volatility:", vol)

20 day volatility: 0.3754370789925649


In [110]:
#go to options page to get strike price and call price
url = "https://sg.finance.yahoo.com/quote/"+ option + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.7)

option_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()

K = float(option_table[9])
#print(option_table)

call_price = float(driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/section[1]/div[2]/div[1]/section/div/section/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

print("Strike_Price:", K)
print("Call_Price:", call_price)

Strike_Price: 140.0
Call_Price: 15.75


In [111]:
#current time
date = datetime.now(tz=pytz.timezone('America/New_York')).replace(microsecond = 0).replace(tzinfo=None)
print("NYC current date and time:", date)

#get time to maturity using 1st row
maturity = datetime.strptime(df['Maturity'].iloc[0], '%Y-%m-%d %H:%M:%S')
time_to_maturity = maturity - date
print("Time to maturity:", time_to_maturity)
#print(time_to_maturity.days, "days", time_to_maturity.seconds, "seconds")

#time to maturity (seconds)
tau_seconds = float(time_to_maturity.days) * 24 * 3600 + float(time_to_maturity.seconds)
#print(tau_seconds, "seconds")

#time to maturity (years)
tau_years = tau_seconds / (365 * 24 * 3600)
#print(tau_years, "years")

NYC current date and time: 2024-10-26 01:47:52
Time to maturity: 83 days, 10:11:08


In [112]:
bsm_price = c(S, K, r, 0, tau_years, vol)
print("Call_Price:", call_price)
print("BSM_Price:", bsm_price)

Call_Price: 15.75
BSM_Price: 10.908277038195521


In [113]:
delta = df['Delta'].iloc[-1]

K = df['Strike_Price'].iloc[0]

new_delta = deltac(S, K, r, 0, tau_years, vol) #vols to be calculated daily

print("Current delta:", delta)
print("New delta:", new_delta)

print("Sell off", delta, "-", new_delta, "shares of stock")

change_in_delta = delta - new_delta
print("Change in delta:", change_in_delta)

Current delta: 0.561263361055655
New delta: 0.5615989907345869
Sell off 0.561263361055655 - 0.5615989907345869 shares of stock
Change in delta: -0.00033562967893185913


In [122]:
previous_date = datetime.strptime(df['Date_Time'].iloc[-1], '%Y-%m-%d %H:%M:%S')
#print("Previous date:", previous_date)

elapsed_days = (date - previous_date).days
elapsed_seconds = (date - previous_date).seconds

elapsed_time_seconds = elapsed_days * 24 * 3600 + elapsed_seconds
#print("Elapsed time (seconds):", elapsed_time_seconds)

elapsed_time_years = elapsed_time_seconds / (365 * 24 * 3600)
#print("Elapsed time (years):", elapsed_time_years)

In [123]:
#profit from selling shares
share_profit = change_in_delta * S
print("Profit from shares:", share_profit)

#amount owed to bank before calculating interest
print("Current amount owed to bank before interest:", df['Amount_Borrowed_from_Bank'].iloc[-1])

#amount owed to bank after calculating interest
amount = df['Amount_Borrowed_from_Bank'].iloc[-1] * np.exp(r * elapsed_time_years)
print("Current amount owed to bank after interest:", amount)

#new amount owed to bank after repaying profit
new_amount = amount - share_profit
print("New amount owed to bank after repaying profit from change in shares:", new_amount)

Profit from shares: -0.04750502475601534
Current amount owed to bank before interest: 63.6912161238174
Current amount owed to bank after interest: 63.69124764911912
New amount owed to bank after repaying profit from change in shares: 63.73875267387513


In [124]:
#wealth
portfolio = new_delta * S - new_amount
print("Value of Replicating Portfolio:", portfolio)

#current profit without hedging
profit_without_hedge = float(df['Call_Price'].iloc[0] - call_price)
print("Profit from closing position without hedge:", profit_without_hedge)

#current profit with hedging
profit = float(format(portfolio - call_price, '.10f'))
print("Profit from closing position with hedge:", profit)

Value of Replicating Portfolio: 15.749968474698285
Profit from closing position without hedge: 0.0
Profit from closing position with hedge: -3.15253e-05


In [118]:
#add new row
row = [date, maturity, S, K, new_amount, r, y, vol, bsm_price, call_price, new_delta, portfolio, profit_without_hedge, profit]

df.loc[len(df.index)] = row
df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,Risk_free_rate,Dividend_Yield,Vol,BSM_price,Call_Price,Delta,Wealth,Profit_without_Hedging,Profit_with_Hedging
0,2024-10-26 00:31:21,2025-01-17 11:59:00,141.54,140.0,63.691216,0.0034,0.0003,0.375437,10.906046,15.75,0.561263,15.750000,0.0,0.000000
1,2024-10-26 01:47:52,2025-01-17 11:59:00,141.54,140.0,63.738870,0.0034,0.0003,0.375437,10.908277,15.75,0.561599,15.749851,0.0,-0.000149
2,2024-10-26 01:47:52,2025-01-17 11:59:00,141.54,140.0,63.738870,0.0034,0.0003,0.375437,10.908277,15.75,0.561599,15.749851,0.0,-0.000149


In [ ]:
from google.colab import files
df.to_csv('NVDA.csv', date_format='%Y-%m-%d %H:%M:%S', index = False)
files.download("NVDA.csv")

#delete last row
# df = df[:-1]
# df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Scripting**

In [ ]:
#connect to Azure SQL server using pyodbc
server = '*****.database.windows.net'
database = '*****'
username = '*****'
password = '*****'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#delete data in table
cursor.execute('''TRUNCATE TABLE dbo.NVDA''')

#execute command
cnxn.commit()

In [ ]:
#export csv file
df.to_csv(r'/home/briansim74/Desktop/NVDA.csv', index = False)

In [ ]:
#BCP utility bulk insert updated CSV file into SQL server
os.system('/opt/mssql-tools18/bin/bcp dbo.NVDA in "/home/briansim74/Desktop/NVDA.csv" -S *****.database.windows.net -d ***** -U ***** -P ***** -c -F 2 -t "," -r "0x0a"')

In [ ]:
#close cursor
cursor.close()

In [ ]:
#quit driver
driver.quit()